In [1]:
import pandas as pd
from transformers import pipeline, set_seed
from sentence_transformers import SentenceTransformer
import requests
from bs4 import BeautifulSoup

In [2]:
picture_info = pd.read_csv('../data/best_picture_2000.csv')

In [3]:
def get_text(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.text)
    # Find all paragraphs within the main content area
    paragraphs = soup.find(id='mw-content-text').find_all('p')
    # Extract plain text from paragraphs using get_text() method
    text = '\n'.join([p.get_text() for p in paragraphs])
    return text
    

In [4]:
picture_info['body_text'] = picture_info['link'].apply(get_text)

In [5]:
picture_info.head()

,title,link,year,body_text
0,Gladiator,https://en.wikipedia.org/wiki/Gladiator_(2000_...,2000,\n\nGladiator is a 2000 historical epic film d...
1,A Beautiful Mind,https://en.wikipedia.org/wiki/A_Beautiful_Mind...,2001,\n\nA Beautiful Mind is a 2001 American biogra...
2,Chicago,https://en.wikipedia.org/wiki/Chicago_(2002_film),2002,\n\nChicago is a 2002 American musical black c...
3,The Lord of the Rings: The Return of the King,https://en.wikipedia.org/wiki/The_Lord_of_the_...,2003,\n\nThe Lord of the Rings: The Return of the K...
4,Million Dollar Baby,https://en.wikipedia.org/wiki/Million_Dollar_Baby,2004,\n\nMillion Dollar Baby is a 2004 American spo...
